In [1]:
%matplotlib inline
import os
import time
import matplotlib
import lasagne
import theano
import theano.tensor as T
import pylab as P
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
from DiscreteLayer import DiscreteLayer
#lasagne.random.set_rng(np.random.RandomState(1234))  # Set random state so we can investigate results
#np.random.seed(123)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 33.0% of memory, cuDNN 5105)
/home/ilker/anaconda2/envs/theano/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:601: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Constants
# theano.config.exception_verbosity = 'high'
conv = lasagne.layers.Conv2DLayer
pool = lasagne.layers.MaxPool2DLayer
NUM_EPOCHS = 1500
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DIM = 60
NUM_CLASSES = 10
mnist_cluttered = "mnist_cluttered_60x60_6distortions.npz"
#DISC
DISC = True
MINS = (.3, -.3, -.8, -.3, .5, -0.7)
MAXS = (1.1, .1, 1.1, .4, 1.1, 1.5)
RANGES = (50, 50, 50, 50, 50, 50)
ADD_NOISE = True
#QUANT = np.array([0.008, 0.004, 0.019, 0.007, 0.006, 0.024], dtype='float32')
QUANT = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1], dtype='float32')
#Test Specs
TH_ACC = 0.98
NUM_TEST = 10
TEST_NAME = 'test1_disc_quantize'

In [3]:
def load_data():
    data = np.load(mnist_cluttered)
    X_train, y_train = data['x_train'], np.argmax(data['y_train'], axis=-1)
    X_valid, y_valid = data['x_valid'], np.argmax(data['y_valid'], axis=-1)
    X_test, y_test = data['x_test'], np.argmax(data['y_test'], axis=-1)

    # reshape for convolutions
    X_train = X_train.reshape((X_train.shape[0], 1, DIM, DIM))
    X_valid = X_valid.reshape((X_valid.shape[0], 1, DIM, DIM))
    X_test = X_test.reshape((X_test.shape[0], 1, DIM, DIM))
    
    print "Train samples:", X_train.shape
    print "Validation samples:", X_valid.shape
    print "Test samples:", X_test.shape

    return dict(
        X_train=lasagne.utils.floatX(X_train),
        y_train=y_train.astype('int32'),
        X_valid=lasagne.utils.floatX(X_valid),
        y_valid=y_valid.astype('int32'),
        X_test=lasagne.utils.floatX(X_test),
        y_test=y_test.astype('int32'),
        num_examples_train=X_train.shape[0],
        num_examples_valid=X_valid.shape[0],
        num_examples_test=X_test.shape[0],
        input_height=X_train.shape[2],
        input_width=X_train.shape[3],
        output_dim=10,)
data = load_data()

Train samples: (50000, 1, 60, 60)
Validation samples: (10000, 1, 60, 60)
Test samples: (10000, 1, 60, 60)


In [4]:
def build_model(input_width, input_height, output_dim, mins, maxs, ranges,
                batch_size=BATCH_SIZE, withdisc=True):
    ini = lasagne.init.HeUniform()
    l_in = lasagne.layers.InputLayer(shape=(None, 1, input_width, input_height),)
    #l_in = lasagne.layers.DropoutLayer(l_in, p=0.1)

    # Localization network
    b = np.zeros((2, 3), dtype=theano.config.floatX)
    b[0, 0] = 1
    b[1, 1] = 1
    b = b.flatten()
    loc_l1 = pool(l_in, pool_size=(2, 2))
    loc_l2 = conv(
        loc_l1, num_filters=20, filter_size=(5, 5), W=ini)
    loc_l3 = pool(loc_l2, pool_size=(2, 2))
    loc_l4 = conv(loc_l3, num_filters=20, filter_size=(5, 5), W=ini)
    loc_l5 = lasagne.layers.DenseLayer(
        loc_l4, num_units=50, W=lasagne.init.HeUniform('relu'))
    loc_out = lasagne.layers.DenseLayer(
        loc_l5, num_units=6, b=b, W=lasagne.init.Constant(0.0), 
        nonlinearity=lasagne.nonlinearities.identity, name='param_regressor')
    
    if withdisc:
        l_dis = DiscreteLayer(loc_out, mins, maxs, ranges, QUANT, addNoise=ADD_NOISE, name='disclayer')
        print("Using Discret. Layer")
    else:
        l_dis = loc_out
        print("No Disc. Layer")
    
    # Transformer network
    l_trans1 = lasagne.layers.TransformerLayer(l_in, l_dis, downsample_factor=3.0)
    print "Transformer network output shape: ", l_trans1.output_shape
    
    # Classification network
    class_l1 = conv(
        l_trans1,
        num_filters=32,
        filter_size=(3, 3),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    )
    class_l2 = pool(class_l1, pool_size=(2, 2))
    class_l3 = conv(
        class_l2,
        num_filters=32,
        filter_size=(3, 3),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    )
    class_l4 = pool(class_l3, pool_size=(2, 2))
    class_l5 = lasagne.layers.DenseLayer(
        class_l4,
        num_units=256,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    )

    l_out = lasagne.layers.DenseLayer(
        class_l5,
        num_units=output_dim,
        nonlinearity=lasagne.nonlinearities.softmax,
        W=ini,
    )

    return l_out, l_trans1

model, l_transform = build_model(DIM, DIM, NUM_CLASSES, MINS, MAXS, RANGES, withdisc=DISC)
model_params = lasagne.layers.get_all_params(model, trainable=True)
params = lasagne.layers.get_all_params(model)

Using Discret. Layer
Transformer network output shape:  (None, 1, 20, 20)


In [5]:
def quantization_devider(dist):
    # Important: dist should be from the epoch at t-1
    # Get the parameters from the network
    quant = params[8] # the quantization parameter as shared
    quant_val = quant.get_value()
    quant_units = quant_val.shape[0]
    changes = []

    # Repeat for each Unit
    for i in range(quant_units):

        # Every quantization number is insterested with one distribution (unit outputs in an epoch)
        T_i = dist[:, i]
        Q = quant_val[i]

        # [Q at t-1] Calculate min, max, range values to obtain the histogram
        T_i_min = np.min(T_i)
        T_i_max = np.max(T_i)
        T_i_bins = np.arange(T_i_min, T_i_max, Q)
        
        # [Q at t-1] Calculate histogram of 
        h_d, _ = np.histogram(T_i, bins=T_i_bins)
        h_d = h_d / float(h_d.sum()) # Convert the histogram into p.density
        h_d_std = np.std(h_d)
        h_d_max = np.max(h_d)
        h_d_nonzero = np.count_nonzero(h_d)
        
        # [Q at t] Calculate min, max, range values to obtain the histogram, mins and maxes are the same
        T_i_bins_t = np.arange(T_i_min, T_i_max, Q*.7)
        
        # [Q at t] Calculate histogram of 
        h_d_t, _ = np.histogram(T_i, bins=T_i_bins_t) # Approximated bins of t over time t-1
        h_d_t = h_d_t / float(h_d_t.sum()) # Convert the histogram into p.density
        h_d_t_max = np.max(h_d_t)
        h_d_std_t = np.std(h_d_t)
        h_d_nonzero_t = np.count_nonzero(h_d_t)

        # Because we want a small std, and make the distribution look uniform
        if  h_d_t_max < h_d_max: # np.prod(h_d_t) > 0:
            Q = Q * .7
            changes.append("idx: {0}, val: {1:.5}".format(i, Q))

        # Change Q
        quant_val[i] = Q
        
    # Report
    if len(changes) > 1:
        print "Changes occured in units: " + ", ".join([str(x) for x in changes] )

    # At the end, update the parameters
    quant.set_value(np.array(quant_val, dtype=theano.config.floatX))

In [6]:
X = T.tensor4(dtype=theano.config.floatX)
y = T.ivector()

## Layer History
if DISC:
    l_disc = next(l for l in lasagne.layers.get_all_layers(model) if l.name is 'disclayer')
    l_paramreg = next(l for l in lasagne.layers.get_all_layers(model) if l.name is 'param_regressor')
    l_disc_output, l_paramreg_output = lasagne.layers.get_output([l_disc, l_paramreg], X, deterministic=False)
## Layer History

# training output
output_train = lasagne.layers.get_output(model, X, deterministic=False)

# evaluation output. Also includes output of transform for plotting
output_eval, transform_eval = lasagne.layers.get_output([model, l_transform], X, deterministic=True)

sh_lr = theano.shared(lasagne.utils.floatX(LEARNING_RATE))
cost = T.mean(T.nnet.categorical_crossentropy(output_train, y))
#updates = lasagne.updates.sgd(cost, model_params, LEARNING_RATE)
updates = lasagne.updates.adam(cost, model_params, learning_rate=sh_lr)

if DISC:
    train = theano.function([X, y], [cost, output_train, l_disc_output, l_paramreg_output], updates=updates)
else:
    train = theano.function([X, y], [cost, output_train], updates=updates)
eval = theano.function([X], [output_eval, transform_eval])

In [7]:
def train_epoch(X, y):
    # History Keeping
    param_output = []
    disc_output = []
    # History
    num_samples = X.shape[0]
    num_batches = int(np.ceil(num_samples / float(BATCH_SIZE)))
    costs = []
    correct = 0
    for i in range(num_batches):
        idx = range(i*BATCH_SIZE, np.minimum((i+1)*BATCH_SIZE, num_samples))
        X_batch = X[idx]
        y_batch = y[idx]
        if DISC:
            cost, output_train, l_disc_output, l_paramreg_output = train(X_batch, y_batch)
            param_output = np.append(param_output, l_paramreg_output)
            disc_output = np.append(disc_output, l_disc_output)
        else:
            cost, output_train = train(X_batch, y_batch)
        costs += [cost]
        preds = np.argmax(output_train, axis=-1)
        correct += np.sum(y_batch == preds)
    
    #import pdb; pdb.set_trace()
    return np.mean(costs), correct / float(num_samples), param_output, disc_output


def eval_epoch(X, y):
    output_eval, transform_eval = eval(X)
    preds = np.argmax(output_eval, axis=-1)
    acc = np.mean(preds == y)
    return acc, transform_eval

# Training

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.4f}'.format}, suppress=True)
valid_accs, train_accs, test_accs = [], [], []
total_time = 0
param_outputs, disc_outputs = [], []
disc_dist_t_1 = None
try:
    for n in range(NUM_EPOCHS):
        start_time = time.time()
        train_cost, train_acc, param_output, disc_output = train_epoch(data['X_train'], data['y_train'])
        valid_acc, valid_trainsform = eval_epoch(data['X_valid'], data['y_valid'])
        test_acc, test_transform = eval_epoch(data['X_test'], data['y_test'])
        valid_accs += [valid_acc]
        test_accs += [test_acc]
        train_accs += [train_acc]

        if DISC:
            param_outputs = np.append(param_outputs, param_output)
            disc_outputs = np.append(disc_outputs, disc_output)

        if (n+1) % 20 == 0:
            new_lr = sh_lr.get_value() * 0.99
            print "New LR:", new_lr
            sh_lr.set_value(lasagne.utils.floatX(new_lr))
        
        # New method to set Quantizations
        if DISC:
            if n>0: # We can only apply this one epoch after
                quantization_devider(disc_dist_t_1)
        disc_dist_t_1 = disc_output.reshape((-1, 6))

        time_spent = time.time() - start_time
        total_time += time_spent
        print "Epoch {0}: T.cost {1:0.6f}, Train acc {2}, val acc {3}, test acc {4}, took {5:.3} sec.".format(
                n, train_cost, train_acc, valid_acc, test_acc, time_spent)

except KeyboardInterrupt:
    pass
print "\nTotal time spent: {0:.5} seconds\nTraing Acc: {1}\nTest Acc: {2}\nValidation Acc: {3}\n".format(total_time,
                                                                                                         train_acc,
                                                                                                         test_acc,
                                                                                                         valid_acc)
story = {'train_accs': train_accs,
         'test_accs': test_accs,
         'valid_accs': valid_accs,
         'epoch_reached': n, 
         'total_time': total_time,
         'disc_enabled': DISC,
         'learning_rate': LEARNING_RATE,
         'batch_size': BATCH_SIZE}                                                                                                         
with open('test_quant_random_uniform_2', 'wb') as fp:
  pickle.dump(story, fp)

Epoch 0: T.cost 1.780250, Train acc 0.36684, val acc 0.6039, test acc 0.6043, took 5.94 sec.
Changes occured in units: idx: 0, val: 0.07, idx: 1, val: 0.07, idx: 3, val: 0.07, idx: 4, val: 0.07
Epoch 1: T.cost 0.998838, Train acc 0.66918, val acc 0.7264, test acc 0.7258, took 5.89 sec.
Epoch 2: T.cost 0.674580, Train acc 0.78354, val acc 0.8173, test acc 0.8124, took 5.91 sec.
Epoch 3: T.cost 0.540250, Train acc 0.82686, val acc 0.8481, test acc 0.8407, took 5.87 sec.
Epoch 4: T.cost 0.465021, Train acc 0.8513, val acc 0.8672, test acc 0.8624, took 5.89 sec.
Changes occured in units: idx: 1, val: 0.049, idx: 5, val: 0.07
Epoch 5: T.cost 0.381446, Train acc 0.87898, val acc 0.8918, test acc 0.8834, took 6.06 sec.
Epoch 6: T.cost 0.343941, Train acc 0.89008, val acc 0.8981, test acc 0.8995, took 5.88 sec.
Changes occured in units: idx: 1, val: 0.0343, idx: 3, val: 0.049
Epoch 7: T.cost 0.308528, Train acc 0.903, val acc 0.9, test acc 0.8973, took 5.95 sec.
Epoch 8: T.cost 0.273434, Train

# Plot Results

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(1-np.array(train_accs), label='Training Error')
plt.plot(1-np.array(valid_accs), label='Validation Error')
plt.legend(fontsize=20)
plt.xlabel('Epoch', fontsize=8)
plt.ylabel('Error', fontsize=8)
plt.show()

# Histograms

In [ ]:
    quant = params[8] # the quantization parameter as shared
    quant_val = quant.get_value()
    print quant_val

In [ ]:
# Histograms
dense_params = param_outputs.reshape((-1, 6))
disc_params = disc_outputs.reshape((-1, 6))

bin_count = 100
plot_over = True

for i in range(0, 6):
    dns = dense_params[:, i]
    
    min_dns = np.min(dense_params[:, i])
    max_dns = np.max(dense_params[:, i])
    bins_dns = np.arange(min_dns, max_dns, QUANT[i])
    
    dsc = disc_params[:, i]
    # print len(np.unique(dsc))
    #PS: Using normed histograms to plot them over
    # Theta x Dense
    plt.figure()
    n, bins, patches = plt.hist(dns, bins='auto', normed=plot_over, histtype='stepfilled')
    plt.setp(patches, 'facecolor', 'r', 'alpha', 0.55)
    if not plot_over:
        plt.xlabel(('Theta({0}) - Discrete Output').format(i+1))
        plt.ylabel('Frequency (Consider bin size)')
        plt.grid(True)
        plt.figure()
    
    # Theta x Discrete
    n, bins, patches = plt.hist(dsc, bins_dns, normed=plot_over, histtype='stepfilled')
    plt.setp(patches, 'facecolor', 'g', 'alpha', 0.55)
    if not plot_over:
        plt.xlabel(('Theta({0}) - Discrete Output').format(i+1))
    else:
        plt.xlabel(('Theta({0})').format(i+1))
    plt.ylabel('Frequency')
    plt.grid(True)


In [ ]:
plt.figure(figsize=(7,14))
for i in range(3):
    plt.subplot(321+i*2)
    plt.imshow(data['X_test'][i].reshape(DIM, DIM), cmap='gray', interpolation='none')
    if i == 0:
        plt.title('Original 60x60', fontsize=20)
    plt.axis('off')
    plt.subplot(322+i*2)
    plt.imshow(test_transform[i].reshape(DIM//3, DIM//3), cmap='gray', interpolation='none')
    if i == 0:
        plt.title('Transformed 20x20', fontsize=20)
    plt.axis('off')
plt.tight_layout()
